In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# metricsa
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score

#xgb
import xgboost as xgb

### Carga de datos

In [2]:
train = pd.read_pickle("../../armado_datos/train_completo")

In [3]:
train["target"] = (train["target"] != 259200.000000) * 1

In [4]:
train.set_index("device_id",inplace = True)

In [14]:
RANDOM_SEED = 12
TEST_SIZE_PERCENT = 0.2

---

### Prueba con todos los features, y ningun nulo

In [3]:
train.dropna(inplace = True)

In [18]:
len(train)

10098

In [19]:
(train["target"] == 0).sum()

1585

In [20]:
(train["target"] == 1).sum()

8513

In [11]:
y = train['target']
X = train.drop(axis=1, labels=['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_PERCENT)

feature_list = list(X.columns)

In [12]:
rdf = xgb.XGBClassifier(learning_rate=0.05, n_estimators=130, max_depth=3,
 min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.6, reg_alpha =0.1)

In [13]:
rdf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.6, gamma=0.3,
       learning_rate=0.05, max_delta_step=0, max_depth=3,
       min_child_weight=5, missing=None, n_estimators=130, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.7, verbosity=1)

In [14]:
ret = rdf.predict_proba(X_test)

In [15]:
y_test_predictd = ret[:, 1]
roc_auc_score(y_test, y_test_predictd)


0.8352490421455939

In [16]:
rdf.score(X_test, y_test)

0.8579207920792079

In [17]:
# Get numerical feature importances
importances = list(rdf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 7)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: apariciones_en_dia_3_x Importance: 0.07833980023860931
Variable: tiempo_ultima_aparicion_hasta_fin_ventana Importance: 0.06392239779233932
Variable: apariciones_en_hora_23 Importance: 0.04261939972639084
Variable: cantidad_apariciones Importance: 0.04084480181336403
Variable: apariciones_en_dia_2_x Importance: 0.033088698983192444
Variable: apariciones_en_dia_3_y Importance: 0.027529200538992882
Variable: tiempo_segunda_hora_frec_aparicion Importance: 0.027157800272107124
Variable: desvio_entre_apariciones Importance: 0.024031000211834908
Variable: tiempo_medio_entre_apariciones Importance: 0.022313300520181656
Variable: tiempo_maximo_entre_apariciones Importance: 0.020145300775766373
Variable: apariciones_en_hora_3 Importance: 0.019584599882364273
Variable: apariciones_en_dia_1_x Importance: 0.018661899492144585
Variable: apariciones_en_hora_19 Importance: 0.01805569976568222
Variable: apariciones_en_source_2 Importance: 0.016830099746584892
Variable: apariciones_en_source_0

---

## Prueba con todos los datos

In [10]:
len(train)

1611351

In [11]:
(train["target"] == 1).sum()

1137784

In [12]:
(train["target"] == 0).sum()

473567

In [6]:
columnas_eliminar = ['apariciones_en_hora_0',
       'apariciones_en_hora_1', 'apariciones_en_hora_2',
       'apariciones_en_hora_3', 'apariciones_en_hora_4',
       'apariciones_en_hora_5', 'apariciones_en_hora_6',
       'apariciones_en_hora_7', 'apariciones_en_hora_8',
       'apariciones_en_hora_9', 'apariciones_en_hora_10',
       'apariciones_en_hora_11', 'apariciones_en_hora_12',
       'apariciones_en_hora_13', 'apariciones_en_hora_14',
       'apariciones_en_hora_15', 'apariciones_en_hora_16',
       'apariciones_en_hora_17', 'apariciones_en_hora_18',
       'apariciones_en_hora_19', 'apariciones_en_hora_20',
       'apariciones_en_hora_21', 'apariciones_en_hora_22',
       'apariciones_en_hora_23',]

In [8]:
train.drop(columns = columnas_eliminar,inplace = True)

In [9]:
train.columns

Index(['target', 'cantidad_apariciones', 'apariciones_en_dia_1_x',
       'apariciones_en_dia_2_x', 'apariciones_en_dia_3_x',
       'cantidad_so_registrados', 'so_media', 'tiempo_medio_entre_apariciones',
       'tiempo_maximo_entre_apariciones', 'desvio_entre_apariciones',
       'apariciones_en_source_0', 'apariciones_en_source_1',
       'apariciones_en_source_2', 'apariciones_en_source_3',
       'apariciones_en_source_4', 'apariciones_en_source_5',
       'apariciones_en_source_6', 'apariciones_en_source_7',
       'apariciones_en_source_8', 'apariciones_en_source_9',
       'tiempo_ultima_aparicion_hasta_fin_ventana',
       'tiempo_hora_frec_aparicion_subasta',
       'tiempo_segunda_hora_frec_aparicion', 'tiempo_hasta_click_mean',
       'tiempo_hasta_click_max', 'tiempo_hasta_click_min',
       'tiempo_hasta_click_std', 'cantidad_clicks', 'apariciones_en_dia_1_y',
       'apariciones_en_dia_2_y', 'apariciones_en_dia_3_y',
       'tiempo_medio_entre_clicks', 'cant_eventos', 'p

In [15]:
y = train['target']
X = train.drop(axis=1, labels=['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_PERCENT)

feature_list = list(X.columns)

In [17]:
rdf = xgb.XGBClassifier(learning_rate=0.05, n_estimators=130, max_depth=3,
 min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.6, reg_alpha =0.1)

In [18]:
rdf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.6, gamma=0.3,
       learning_rate=0.05, max_delta_step=0, max_depth=3,
       min_child_weight=5, missing=None, n_estimators=130, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.7, verbosity=1)

In [19]:
ret = rdf.predict_proba(X_test)

In [20]:
y_test_predictd = ret[:, 1]
roc_auc_score(y_test, y_test_predictd)


0.7607191256961423

In [21]:
rdf.score(X_test, y_test)

0.7261776579338507

## Prueba solo datos de auctions sin los datos de cada hora

In [24]:
train.columns

Index(['target', 'cantidad_apariciones', 'apariciones_en_dia_1_x',
       'apariciones_en_dia_2_x', 'apariciones_en_dia_3_x',
       'cantidad_so_registrados', 'so_media', 'tiempo_medio_entre_apariciones',
       'tiempo_maximo_entre_apariciones', 'desvio_entre_apariciones',
       'apariciones_en_source_0', 'apariciones_en_source_1',
       'apariciones_en_source_2', 'apariciones_en_source_3',
       'apariciones_en_source_4', 'apariciones_en_source_5',
       'apariciones_en_source_6', 'apariciones_en_source_7',
       'apariciones_en_source_8', 'apariciones_en_source_9',
       'tiempo_ultima_aparicion_hasta_fin_ventana',
       'tiempo_hora_frec_aparicion_subasta',
       'tiempo_segunda_hora_frec_aparicion', 'tiempo_hasta_click_mean',
       'tiempo_hasta_click_max', 'tiempo_hasta_click_min',
       'tiempo_hasta_click_std', 'cantidad_clicks', 'apariciones_en_dia_1_y',
       'apariciones_en_dia_2_y', 'apariciones_en_dia_3_y',
       'tiempo_medio_entre_clicks', 'cant_eventos', 'p

In [26]:
columnas_eliminar = ['tiempo_hasta_click_mean',
       'tiempo_hasta_click_max', 'tiempo_hasta_click_min',
       'tiempo_hasta_click_std', 'cantidad_clicks', 'apariciones_en_dia_1_y',
       'apariciones_en_dia_2_y', 'apariciones_en_dia_3_y',
       'tiempo_medio_entre_clicks', 'cant_eventos', 'prom_eventos',
       'eventos_dia_1', 'eventos_dia_2', 'eventos_dia_3', 'max_cant_eventos',
       'cant_eventos_maximos', 'segs_hasta_hora_freq', 'eventos_franja_1',
       'eventos_franja_2', 'eventos_franja_3', 'eventos_franja_4',
       'eventos_franja_5', 'eventos_franja_6', 'cant_eventos_wifi',
       'cant_eventos_sin_wifi', 'cant_eventos_15', 'cant_eventos_23',
       'cant_eventos_7', 'cant_eventos_0', 'cant_eventos_2']

In [28]:
train.drop(columns = columnas_eliminar,inplace = True)

In [30]:
train.isnull().sum().sum()

0

In [31]:
len(train)

1611351

In [32]:
y = train['target']
X = train.drop(axis=1, labels=['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_PERCENT)

feature_list = list(X.columns)

In [33]:
rdf = xgb.XGBClassifier(learning_rate=0.05, n_estimators=130, max_depth=3,
 min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.6, reg_alpha =0.1)

rdf.fit(X_train, y_train)

ret = rdf.predict_proba(X_test)


In [34]:
y_test_predictd = ret[:, 1]
roc_auc_score(y_test, y_test_predictd)


0.7607431674054499

In [35]:
rdf.score(X_test, y_test)

0.7259573464568639

### Prueba con los features de events y auctions sin nulos

In [2]:
train = pd.read_pickle("../../armado_datos/train_completo")
train["target"] = (train["target"] != 259200.000000) * 1
train.set_index("device_id",inplace = True)


In [3]:
RANDOM_SEED = 12
TEST_SIZE_PERCENT = 0.2

In [4]:
train.columns

Index(['target', 'cantidad_apariciones', 'apariciones_en_hora_0',
       'apariciones_en_hora_1', 'apariciones_en_hora_2',
       'apariciones_en_hora_3', 'apariciones_en_hora_4',
       'apariciones_en_hora_5', 'apariciones_en_hora_6',
       'apariciones_en_hora_7', 'apariciones_en_hora_8',
       'apariciones_en_hora_9', 'apariciones_en_hora_10',
       'apariciones_en_hora_11', 'apariciones_en_hora_12',
       'apariciones_en_hora_13', 'apariciones_en_hora_14',
       'apariciones_en_hora_15', 'apariciones_en_hora_16',
       'apariciones_en_hora_17', 'apariciones_en_hora_18',
       'apariciones_en_hora_19', 'apariciones_en_hora_20',
       'apariciones_en_hora_21', 'apariciones_en_hora_22',
       'apariciones_en_hora_23', 'apariciones_en_dia_1_x',
       'apariciones_en_dia_2_x', 'apariciones_en_dia_3_x',
       'cantidad_so_registrados', 'so_media', 'tiempo_medio_entre_apariciones',
       'tiempo_maximo_entre_apariciones', 'desvio_entre_apariciones',
       'apariciones_en_sou

In [5]:
columnas_clicks = ['tiempo_hasta_click_mean',
       'tiempo_hasta_click_max', 'tiempo_hasta_click_min',
       'tiempo_hasta_click_std', 'cantidad_clicks', 'apariciones_en_dia_1_y',
       'apariciones_en_dia_2_y', 'apariciones_en_dia_3_y',
       'tiempo_medio_entre_clicks']

In [6]:
train.drop(columns = columnas_clicks, inplace= True)

In [7]:
train.dropna(inplace = True)

In [8]:
len(train)

401576

In [9]:
y = train['target']
X = train.drop(axis=1, labels=['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_PERCENT)

feature_list = list(X.columns)

In [10]:
model = xgb.XGBClassifier(learning_rate=0.05, n_estimators=130, max_depth=3,
 min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.6, reg_alpha =0.1)

model.fit(X_train, y_train)

ret = model.predict_proba(X_test)


In [11]:
y_test_predictd = ret[:, 1]
roc_auc_score(y_test, y_test_predictd)

0.7733783329058075

In [13]:
model.score(X_test, y_test)

0.7298047711539419

In [14]:
model.save_model("0001.modelo_clasificacion")

In [13]:
y_test_predictd = ret[:, 1]
roc_auc_score(y_test, y_test_predictd)


0.7713416452975753

In [14]:
rdf.score(X_test, y_test)

0.7301782957318591

In [15]:
# Get numerical feature importances
importances = list(rdf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 7)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: desvio_entre_apariciones Importance: 0.26705020666122437
Variable: tiempo_maximo_entre_apariciones Importance: 0.09076780080795288
Variable: apariciones_en_dia_3_x Importance: 0.07831910252571106
Variable: tiempo_ultima_aparicion_hasta_fin_ventana Importance: 0.04868759959936142
Variable: apariciones_en_dia_2_x Importance: 0.04182739928364754
Variable: tiempo_medio_entre_apariciones Importance: 0.03817790001630783
Variable: apariciones_en_hora_2 Importance: 0.02842009998857975
Variable: cantidad_apariciones Importance: 0.028287900611758232
Variable: apariciones_en_dia_1_x Importance: 0.02703920006752014
Variable: apariciones_en_hora_23 Importance: 0.02547050081193447
Variable: apariciones_en_hora_19 Importance: 0.025250399485230446
Variable: apariciones_en_hora_22 Importance: 0.023350799456238747
Variable: prom_eventos         Importance: 0.01897530071437359
Variable: cant_eventos_sin_wifi Importance: 0.01680069975554943
Variable: tiempo_segunda_hora_frec_aparicion Importance

### Prueba con todos los datos habidos y por haber

In [3]:
train = pd.read_pickle("../../armado_datos/train_completo")
train["target"] = (train["target"] != 259200.000000) * 1
train.set_index("device_id",inplace = True)


In [4]:
RANDOM_SEED = 12
TEST_SIZE_PERCENT = 0.2

In [6]:
y = train['target']
X = train.drop(axis=1, labels=['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_PERCENT)

feature_list = list(X.columns)

In [7]:
rdf = xgb.XGBClassifier(learning_rate=0.05, n_estimators=130, max_depth=3,
 min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.6, reg_alpha =0.1)

rdf.fit(X_train, y_train)

ret = rdf.predict_proba(X_test)

In [8]:
y_test_predictd = ret[:, 1]
roc_auc_score(y_test, y_test_predictd)


0.7601573213008006

In [9]:
rdf.score(X_test, y_test)

0.7254639728675556

In [10]:
# Get numerical feature importances
importances = list(rdf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 7)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: desvio_entre_apariciones Importance: 0.25004830956459045
Variable: tiempo_maximo_entre_apariciones Importance: 0.11683730036020279
Variable: apariciones_en_dia_3_x Importance: 0.0799700990319252
Variable: tiempo_medio_entre_apariciones Importance: 0.07605639845132828
Variable: tiempo_ultima_aparicion_hasta_fin_ventana Importance: 0.051513999700546265
Variable: apariciones_en_dia_2_x Importance: 0.04676060006022453
Variable: cantidad_apariciones Importance: 0.04616620019078255
Variable: apariciones_en_hora_23 Importance: 0.030471399426460266
Variable: apariciones_en_dia_1_x Importance: 0.026668399572372437
Variable: apariciones_en_hora_21 Importance: 0.01733729988336563
Variable: tiempo_segunda_hora_frec_aparicion Importance: 0.017138900235295296
Variable: apariciones_en_hora_17 Importance: 0.016501199454069138
Variable: apariciones_en_hora_16 Importance: 0.014976000413298607
Variable: apariciones_en_hora_20 Importance: 0.014663799665868282
Variable: apariciones_en_hora_18 Imp

## Entreno un clasficador que utilice solo datos de auctions

In [3]:
train = pd.read_pickle("../../armado_datos/train_completo")
train["target"] = (train["target"] != 259200.000000) * 1
train.set_index("device_id",inplace = True)


In [4]:
RANDOM_SEED = 12
TEST_SIZE_PERCENT = 0.2

#### Elimino los datos de los dispositivos que tienen datos para eventos

In [5]:
train = train[ ~(train["cant_eventos"].notna())]

In [7]:
columnas_auctions = ['target', 'cantidad_apariciones', 'apariciones_en_hora_0',
       'apariciones_en_hora_1', 'apariciones_en_hora_2',
       'apariciones_en_hora_3', 'apariciones_en_hora_4',
       'apariciones_en_hora_5', 'apariciones_en_hora_6',
       'apariciones_en_hora_7', 'apariciones_en_hora_8',
       'apariciones_en_hora_9', 'apariciones_en_hora_10',
       'apariciones_en_hora_11', 'apariciones_en_hora_12',
       'apariciones_en_hora_13', 'apariciones_en_hora_14',
       'apariciones_en_hora_15', 'apariciones_en_hora_16',
       'apariciones_en_hora_17', 'apariciones_en_hora_18',
       'apariciones_en_hora_19', 'apariciones_en_hora_20',
       'apariciones_en_hora_21', 'apariciones_en_hora_22',
       'apariciones_en_hora_23', 'apariciones_en_dia_1_x',
       'apariciones_en_dia_2_x', 'apariciones_en_dia_3_x',
       'cantidad_so_registrados', 'so_media', 'tiempo_medio_entre_apariciones',
       'tiempo_maximo_entre_apariciones', 'desvio_entre_apariciones',
       'apariciones_en_source_0', 'apariciones_en_source_1',
       'apariciones_en_source_2', 'apariciones_en_source_3',
       'apariciones_en_source_4', 'apariciones_en_source_5',
       'apariciones_en_source_6', 'apariciones_en_source_7',
       'apariciones_en_source_8', 'apariciones_en_source_9',
       'tiempo_ultima_aparicion_hasta_fin_ventana',
       'tiempo_hora_frec_aparicion_subasta',
       'tiempo_segunda_hora_frec_aparicion']

train = train[columnas_auctions]

In [8]:
len(train)

1209775

In [13]:
y = train['target']
X = train.drop(axis=1, labels=['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_PERCENT)

feature_list = list(X.columns)

In [14]:
rdf = xgb.XGBClassifier(learning_rate=0.05, n_estimators=130, max_depth=3,
 min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.6, reg_alpha =0.1)

rdf.fit(X_train, y_train)

ret = rdf.predict_proba(X_test)


In [15]:
y_test_predictd = ret[:, 1]
roc_auc_score(y_test, y_test_predictd)


0.756545593308659

In [16]:
rdf.score(X_test, y_test)

0.7249033911264491

In [17]:
rdf.save_model("0002.modelo_clasificacion")

In [18]:
# Get numerical feature importances
importances = list(rdf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 7)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: desvio_entre_apariciones Importance: 0.2494467943906784
Variable: tiempo_maximo_entre_apariciones Importance: 0.15351399779319763
Variable: apariciones_en_dia_3_x Importance: 0.09283290058374405
Variable: tiempo_ultima_aparicion_hasta_fin_ventana Importance: 0.06073259934782982
Variable: cantidad_apariciones Importance: 0.056953199207782745
Variable: apariciones_en_dia_1_x Importance: 0.038067299872636795
Variable: apariciones_en_dia_2_x Importance: 0.03595050051808357
Variable: apariciones_en_hora_23 Importance: 0.029658600687980652
Variable: tiempo_medio_entre_apariciones Importance: 0.026326900348067284
Variable: tiempo_segunda_hora_frec_aparicion Importance: 0.02101230062544346
Variable: apariciones_en_hora_19 Importance: 0.01557410042732954
Variable: apariciones_en_hora_20 Importance: 0.015040099620819092
Variable: apariciones_en_hora_22 Importance: 0.014982500113546848
Variable: apariciones_en_hora_14 Importance: 0.014196000061929226
Variable: apariciones_en_hora_15 Imp

## Entreno con la mitad de todos los datos disponibles

In [2]:
train = pd.read_pickle("../../armado_datos/train_completo")
#train["target"] = (train["target"] != 259200.000000) * 1
train.set_index("device_id",inplace = True)


---

### Separo la mitad de los datos para el otro modelo

In [3]:
y = train['target']
X = train.drop(axis=1, labels=['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [4]:
X_test.to_pickle("train_modelo_regresion")

In [5]:
y_test.to_pickle("target_modelo_regresion")

---

In [6]:
y_train = (y_train != 259200.000000) * 1


In [8]:
y = y_train
X = X_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
rdf = xgb.XGBClassifier(learning_rate=0.05, n_estimators=130, max_depth=3,
 min_child_weight=5, gamma=0.3, subsample=0.7, colsample_bytree=0.6, reg_alpha =0.1)

rdf.fit(X_train, y_train)

ret = rdf.predict_proba(X_test)

In [11]:
y_test_predictd = ret[:, 1]
roc_auc_score(y_test, y_test_predictd)


0.7592156149045788

In [14]:
rdf.score(X_test, y_test)

0.7265150339777205

In [15]:
rdf.save_model("0003.modelo_clasificacion")